# Urban Water Model Simulation for Fehraltorf

This notebook runs the urban water model simulation for Fehraltorf and visualizes the results.

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import geopandas as gpd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from shapely.geometry import LineString
import numpy as np

from IPython.display import display
import ipywidgets as widgets

from duwcm import run
from duwcm.functions import load_config

# Functions

In [ ]:
def plot_results(aggregated_results, forcing):
    plot_configs = [
        'Evapotranspiration',
        'Stormwater',
        'Baseflow',
        'Wastewater'
    ]
    
    # Define specific colors for each variable
    variable_colors = {
        'Evapotranspiration': 'green',
        'Stormwater': 'orange',
        'Baseflow': 'red',
        'Wastewater': 'brown'
    }
    
    total_area = aggregated_results.attrs['total_area']
    units = aggregated_results.attrs['units']
    
    plot_data = pd.DataFrame({
        'Precipitation': forcing['precipitation'],
        'Evaporation': forcing['potential_evaporation'],
        'Evapotranspiration': (aggregated_results['evaporation'] + aggregated_results['transpiration']) / total_area,
        'Stormwater': aggregated_results['stormwater'] / total_area,
        'Baseflow': aggregated_results['baseflow'] / total_area,
        'Wastewater': aggregated_results['wastewater'] / total_area
    })
    
    fig = go.Figure()

    for i, config in enumerate(plot_configs):
        fig.add_trace(go.Scatter(
            x=plot_data.index,
            y=plot_data['Precipitation'],
            name='Precipitation',
            fill='tozeroy',
            line=dict(color='lightblue'),
            visible=False
        ))
        if config == 'Evapotranspiration':
            fig.add_trace(go.Scatter(
                x=plot_data.index,
                y=plot_data['Evaporation'],
                name='Potential Evaporation',
                line=dict(color='purple', dash='dash'),
                yaxis='y2',
                visible=False
            ))
            fig.add_trace(go.Scatter(
                x=plot_data.index,
                y=plot_data[config],
                name=config,
                line=dict(color=variable_colors[config]),
                yaxis='y2',
                visible=False
            ))
        else:
            fig.add_trace(go.Scatter(
                x=plot_data.index,
                y=plot_data['Evapotranspiration'],
                name='Evapotranspiration',
                line=dict(color='green', dash='dash'),
                visible=False
            ))
            fig.add_trace(go.Scatter(
                x=plot_data.index,
                y=plot_data[config],
                name=config,
                line=dict(color=variable_colors[config]),
                yaxis='y2',
                visible=False
            ))


    # Make the first set of traces visible
    for i in range(3):
        fig.data[i].visible = True

    dropdown_buttons = []
    for i, config in enumerate(plot_configs):
        visibility = [False] * len(fig.data)
        for j in range(3):
            visibility[i*3 + j] = True
        dropdown_buttons.append(
            dict(
                label=config,
                method='update',
                args=[{
                    'visible': visibility
                },
                {
                    'yaxis.title.text': 'Precipitation & Evapotranspiration [mm/day]',
                    'yaxis2.title.text': f"{config} [mm/day]"
                }]
            )
        )

    fig.update_layout(
        updatemenus=[dict(
            active=0,
            buttons=dropdown_buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.15,
            yanchor="top"
        )],
        yaxis=dict(title='Precipitation & Evapotranspiration [mm/day]'),
        yaxis2=dict(overlaying='y', side='right'),
        height=600,
        hovermode='x unified',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    return fig.show()

In [ ]:
# Plotly configuration for notebook
def create_global_map(geometry_geopackage: Path, background_shapefile: Path):
    # Read the geometry data
    gdf_geometry = gpd.read_file(geometry_geopackage)
    
    # Read the background shapefile
    gdf_background = gpd.read_file(background_shapefile)
    
    # Ensure both GeoDataFrames use the same CRS
    if gdf_geometry.crs != gdf_background.crs:
        gdf_background = gdf_background.to_crs(gdf_geometry.crs)
    
    # Project to Web Mercator for accurate centroid calculation
    gdf_background_projected = gdf_background.to_crs(epsg=3857)
    
    # Calculate the centroid in the projected CRS
    centroid_projected = gdf_background_projected.geometry.centroid
    
    # Convert the centroid back to WGS84
    centroid = centroid_projected.to_crs(epsg=4326)
    
    # Convert geometries to EPSG:4326 for Plotly
    gdf_geometry = gdf_geometry.to_crs(epsg=4326)
    gdf_background = gdf_background.to_crs(epsg=4326)
    
    # Create the figure
    fig = go.Figure()
    
    # Add the hexagons with very low opacity fill and thin borders
    fig.add_trace(go.Choroplethmapbox(
        geojson=gdf_geometry.__geo_interface__,
        locations=gdf_geometry.index,
        z=[1]*len(gdf_geometry),
        colorscale=['white', 'lightblue'],
        showscale=False,
        marker_opacity=0.5,
        marker_line_width=0.6,
        marker_line_color='lightblue'
    ))
    
    # Function to extract coordinates from a polygon
    def get_polygon_coordinates(polygon):
        if polygon.geom_type == 'Polygon':
            return [list(polygon.exterior.coords)]
        elif polygon.geom_type == 'MultiPolygon':
            return [list(geom.exterior.coords) for geom in polygon.geoms]
        else:
            return []

    # Add the background as the contour (thick line, no fill)
    for _, row in gdf_background.iterrows():
        polygons = get_polygon_coordinates(row.geometry)
        for polygon in polygons:
            lons, lats = zip(*polygon)
            fig.add_trace(go.Scattermapbox(
                lon=list(lons),
                lat=list(lats),
                mode='lines',
                line=dict(width=1, color='gray'),
                showlegend=False
            ))
    
    # Calculate the bounding box of the background
    bounds = gdf_background.total_bounds
    center_lon, center_lat = (bounds[0] + bounds[2]) / 2, (bounds[1] + bounds[3]) / 2

    fig.update_layout(
        mapbox_style="carto-positron",
        mapbox=dict(
            bearing=0,
            center=dict(lat=center_lat, lon=center_lon),
            pitch=0,
            zoom=10
        ),
        margin={"r":0,"t":0,"l":0,"b":0}
    )
    
    return fig

In [ ]:
def create_area_map(gdf, variables, time_series_data):
    # Convert to EPSG:4326 for Plotly
    gdf_wgs84 = gdf.to_crs(epsg=4326)
    
    # Calculate the center of the map using bounds
    bounds = gdf_wgs84.total_bounds
    center_lon = (bounds[0] + bounds[2]) / 2
    center_lat = (bounds[1] + bounds[3]) / 2

    # Resample the time series data to monthly frequency
    monthly_data = time_series_data.groupby(pd.Grouper(freq='ME')).mean()

    # Calculate global min and max for each variable
    global_min = {var: monthly_data[var].min().min() for var in variables}
    global_max = {var: monthly_data[var].max().max() for var in variables}

    # Create the base figure
    fig = go.Figure()

    # Add a trace for each variable
    for variable in variables:
        fig.add_trace(go.Choroplethmapbox(
            geojson=gdf_wgs84.__geo_interface__,
            locations=gdf_wgs84.index,
            z=monthly_data.loc[monthly_data.index[0], variable],
            zmin=global_min[variable],
            zmax=global_max[variable],
            colorscale="Viridis",
            marker_opacity=0.7,
            marker_line_width=0,
            colorbar_title_text=f"{variable.replace('_', ' ').capitalize()} [ML/yr]",
            colorbar_title_side="right",
            colorbar_thickness=20,
            colorbar_title_font=dict(size=12),
            visible=variable == variables[0],
            name=variable
        ))

    # Create slider
    sliders = [dict(
        active=0,
        currentvalue={"prefix": "Month: ", "offset": 20},
        pad={"b": 10, "t": 50},
        len=0.875,
        x=0.0625,
        xanchor="left",
        y=0,
        yanchor="top",
        steps=[dict(
            method='update',
            args=[{'z': [monthly_data.loc[month_end, var] for var in variables]}],
            label=month_end.strftime('%Y-%m')
        ) for month_end in monthly_data.index]
    )]

    # Create and add dropdown menu
    dropdown_buttons = []
    for i, variable in enumerate(variables):
        dropdown_buttons.append(dict(
            args=[{"visible": [i == j for j in range(len(variables))],
                   "colorbar_title_text": f"{variable.replace('_', ' ').capitalize()} [ML/yr]"}],
            label=variable.replace('_', ' ').capitalize(),
            method="update"
        ))

    # Update layout
    fig.update_layout(
        mapbox_style="carto-positron",
        mapbox=dict(
            center=dict(lat=center_lat, lon=center_lon),
            zoom=11
        ),
        updatemenus=[
            dict(
                buttons=dropdown_buttons,
                direction="down",
                pad={"r": 10, "t": 10},
                showactive=True,
                x=0.1,
                xanchor="left",
                y=1.1,
                yanchor="top"
            )
        ],
        sliders=sliders,
        margin={"r":0, "t":45, "l":0, "b":120},
        height=700
    )

    return fig

## Configuration

Set up the configuration for the simulation.

In [ ]:
# Load configuration
config_path = 'config.toml'
config = load_config(config_path, env='default')

# Display some key configuration settings
print(f"Input directory: {config.input_directory}")
print(f"Output directory: {config.output.output_directory}")
print(f"Simulation period: {config.simulation.start_year} - {config.simulation.end_year}")

# Get the paths to the GeoPackage and background shapefile
geo_file = Path(config.input_directory) / Path(config.files.geo_file)
background_file = Path(config.geodata_directory) / config.files.background_shapefile


# Create global map
global_map = create_global_map(geo_file, background_file)
display(global_map)

## Run Simulation

Execute the urban water model simulation.

In [ ]:
# Run the simulation
results, model_params, forcing_data, flow_paths = run(config)

## Visualize Results

Create plots to visualize the simulation results.

In [ ]:
# Plot aggregated results
fig = plot_results(results['aggregated'], forcing_data)

## Generate Interactive Maps

Create interactive map visualizations of the results using Folium.

In [ ]:
# Prepare data for mapping
area_variables = [
    'evapotranspiration',
    'imported_water',
    'baseflow',
    'deep_seepage',
    'stormwater_runoff',
    'wastewater_runoff'
]

gdf_geometry = gpd.read_file(geo_file)

# Prepare time series data
time_series_data = results['local'][area_variables].unstack(level='cell')

# Convert the index to datetime if it's not already
time_series_data.index = pd.to_datetime(time_series_data.index)

# Create and display the interactive monthly dynamic area map
area_map = create_area_map(gdf_geometry, area_variables, time_series_data)
area_map.show()